# Exploratory Data Analysis — Bike Sharing Dataset

**Goal**: Understand patterns in hourly bike rentals (`cnt`) to support building a daily prediction service model for planning processes and bicycle logistics.

Dataset source: [UCI Bike Sharing Dataset](https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset)


In [ ]:
# import packages
import pandas as pd
import numpy as np
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.graph_objects as go
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from utils_nb import *

In [ ]:
def create_and_show_fig(fig, title):
    """Helper function to create and show figures with consistent layout."""
    fig.update_layout(width=600, height=350, title=title)  # Set figure size here
    fig.show()

---
## 1. Load & Preview Data

In [ ]:
df = pd.read_csv("../data/hour.csv")  # 'hour.csv' is the hourly dataset
print(df.shape)
df.head(2)

In [ ]:
# data status, columns, dtypes, non-null count
df.info()

## 2. Basic Cleaning & Preprocessing

### 2.1. Convert 'dteday' to datetime

In [ ]:
# Convert date column
df['dteday'] = pd.to_datetime(df['dteday'])

### 2.2. Check  missing values

In [ ]:
# Check for missing values
missing_values = df.isnull().values.any()
print(f"Missing values in dataset: {missing_values}")

### 2.3 Check Dataset Coverage & Completeness
- A few days contain fewer than 24 records, indicating data incompleteness.

In [ ]:
# Check the overall date coverage
print("Date Range:")
print(f"Start: {df['dteday'].min().date()}  |  End: {df['dteday'].max().date()}")

# Count number of unique days
num_days = df['dteday'].nunique()
print(f"Total unique days: {num_days}")

# Show available years
years = df['dteday'].dt.year.unique()
print(f"Years in dataset: {sorted(years)}")

# Check how many records per day (should be 24 ideally)
daily_record_unique_counts = df.groupby('dteday')['instant'].count().value_counts()
print("\nUnique records per day:")
print(daily_record_unique_counts)
print("\n")

# Check duplicates dates and hours
dupli_num = df.duplicated(subset=['dteday', 'hr'], keep='last').shape[0] - df.shape[0]
print(f"Duplicated records number: {dupli_num}")

# Check cnt=0 records
zero_cnt_num = df[df['cnt'] == 0].shape[0]
print(f"Zero cnt records number: {zero_cnt_num}")

In [ ]:
# Clean and fill missing hours
df_cleaned = fill_missing_hours(df)

# Confirm daily completeness
assert all(df_cleaned.groupby('dteday')['cnt'].count() == 24)

# Confirm no nulls left
print(df_cleaned.isnull().any().any())

# Preview cleaned data
df_cleaned.head(2)

---
## 2. Statistics Analysis

In [ ]:
# Target Variable (cnt)
fig_cnt_density = px.histogram(df_cleaned, x='cnt', marginal='rug', title = f'Distribution & Density plot of cnt')
fig_cnt_box = px.box(df_cleaned, y="cnt", title="Box plot of cnt")
create_and_show_fig(fig_cnt_density, 'Distribution & Density plot of cnt')
create_and_show_fig(fig_cnt_box, title="Box plot of cnt")


In [ ]:
# Numerical Features
numerical_features = ['temp', 'atemp', 'hum', 'windspeed']
for feature in numerical_features:
    fig_box = px.box(df_cleaned, y=feature, title=f"Box plot of {feature}")
    fig_density = px.histogram(df_cleaned, x=feature, marginal='rug', title = f'Distribution & Density plot of {feature}')
    create_and_show_fig(fig_density, f"Distribution & Density plot of {feature}")
    create_and_show_fig(fig_box, f"Box plot of {feature}")

In [ ]:
# Categorical Features
# The distribution of ['season', 'yr', 'mnth', 'hr', 'weekday'] is straightforward, ignore here
categorical_features =  ['workingday', 'weathersit']
for feature in categorical_features:
    fig_bar = px.bar(df_cleaned[feature].value_counts().sort_index(), title=f"Count of {feature}")
    create_and_show_fig(fig_bar, f"Count of {feature}")

In [ ]:
# 3. Bivariate Analysis
# Target vs. Numerical
numerical_features = ['temp', 'atemp', 'hum', 'windspeed']
for feature in numerical_features:
    fig_scatter = px.scatter(df_cleaned, x=feature, y="cnt", title=f"cnt vs. {feature}")
    create_and_show_fig(fig_scatter, f"cnt vs. {feature}")

fig_line_hr = px.line(df_cleaned.groupby('hr')['cnt'].mean().reset_index(), x='hr', y='cnt', title = 'Mean cnt by hour')
create_and_show_fig(fig_line_hr, 'Mean cnt by hour')

In [ ]:
# Target vs. Categorical
categorical_features = ['season', 'yr', 'mnth', 'weekday', 'workingday', 'weathersit']
for feature in categorical_features:
    fig_box_cat = px.box(df_cleaned, x=feature, y="cnt", title=f"cnt vs. {feature}")
    create_and_show_fig(fig_box_cat, f"cnt vs. {feature}")
    fig_bar_mean = px.bar(df_cleaned.groupby(feature)['cnt'].mean(), title = f'Mean cnt by {feature}')
    create_and_show_fig(fig_bar_mean, f'Mean cnt by {feature}')

In [ ]:
# Correlation Analysis
correlation_matrix = df_cleaned[numerical_features + ['cnt']].corr()
fig_heatmap = px.imshow(correlation_matrix, title="Correlation Matrix")
fig_heatmap.show()

In [ ]:
# 4. Time Series Analysis
fig_time_series = px.line(df_cleaned, x='dteday', y='cnt', title='cnt over time')
create_and_show_fig(fig_time_series, "cnt over time")

In [ ]:
df_cleaned['dteday'] = pd.to_datetime(df_cleaned['dteday'])
# Use resample to set the frequency and aggregate to daily sums
daily_cnt = df_cleaned.set_index('dteday')['cnt'].resample('D').sum() #set index, and resample.

# Seasonal Decomposition
decomposition = seasonal_decompose(daily_cnt, model="additive", period=30)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Plotting the Decomposition
fig_original = px.line(x=daily_cnt.index, y=daily_cnt, title="Original Time Series")
create_and_show_fig(fig_original, "Original Time Series")

fig_trend = px.line(x=trend.index, y=trend, title="Trend Component")
create_and_show_fig(fig_trend, "Trend Component")

fig_seasonal = px.line(x=seasonal.index, y=seasonal, title="Seasonal Component")
create_and_show_fig(fig_seasonal, "Seasonal Component")

fig_residual = px.line(x=residual.index, y=residual, title="Residual Component")
create_and_show_fig(fig_residual, "Residual Component")

In [ ]:
# Plot ACF and PACF
plt.figure(figsize=(12, 6))

# ACF plot
plt.subplot(211)
plot_acf(daily_cnt, lags=730, ax=plt.gca()) #lags set to 365, to show a years worth of data.
plt.title('Autocorrelation Function (ACF)')

# PACF plot
plt.subplot(212)
plot_pacf(daily_cnt, lags=365,ax=plt.gca()) #lags set to 30, to show a month worth of data.
plt.title('Partial Autocorrelation Function (PACF)')

plt.tight_layout()
plt.show()

In [ ]:
hourly_cnt = df_cleaned.groupby("datetime").cnt.sum()

# Apply Differencing
hourly_diff = hourly_cnt.diff(24).dropna()

check_stationarity(hourly_cnt)
check_stationarity(hourly_diff)



In [ ]:
# Plot ACF and PACF
plt.figure(figsize=(12, 6))

# ACF plot
plt.subplot(211)
plot_acf(hourly_cnt, lags=24, ax=plt.gca()) #lags set to 365, to show a years worth of data.
plt.title('Autocorrelation Function (ACF)')

# PACF plot
plt.subplot(212)
plot_pacf(hourly_cnt, lags=24,ax=plt.gca()) #lags set to 30, to show a month worth of data.
plt.title('Partial Autocorrelation Function (PACF)')

plt.tight_layout()
plt.show()

In [ ]:
# Plot ACF and PACF using Plotly

plot_acf_pacf_plotly(hourly_cnt, lags=72)

In [ ]:
# Plot ACF and PACF using Plotly

plot_acf_pacf_plotly(hourly_diff, lags=72)


In [ ]:
test_weeks = [
    ("2012-08-01", "2012-08-07"),
    ("2012-12-25", "2012-12-31")
]

In [ ]:
# Run all models
baseline_df = evaluate_baseline(hourly_cnt, test_weeks)
arima_df = evaluate_arima(hourly_cnt, test_weeks)
xgb_df = evaluate_xgboost_weekly(df_cleaned, test_weeks)


In [ ]:
results = pd.concat([arima_df, xgb_df,baseline_df])
results = label_test_period(results, test_weeks)
results['date'] = pd.to_datetime(results['date'])
results = results.sort_values(by=["model", "date"])


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get the unique test periods
test_periods = results['test_period'].unique()

model_colors = {
    'ARIMA': '#636EFA',
    'Baseline': '#EF553B',
    'XGBoost': '#00CC96'
}

# Create subplots (1 row per test week)
fig = make_subplots(
    rows=len(test_periods),
    cols=1,
    shared_xaxes=False,
    subplot_titles=test_periods
)

for i, period in enumerate(test_periods):
    df_sub = results[results['test_period'] == period]
    models = df_sub['model'].unique()

    for model in models:
        df_model = df_sub[df_sub['model'] == model]
        fig.add_trace(
            go.Scatter(
                x=df_model['date'],
                y=df_model['mae'],
                mode='lines+markers',
                name=model,
                legendgroup=model,
                showlegend=(i == 0),  # only show legend once
                line=dict(color=model_colors.get(model, 'gray'))  # << add this
            ),
            row=i+1,
            col=1
        )

# Update layout
fig.update_layout(
    height=500 * len(test_periods),
    title_text="Model MAE Over Selected Test Weeks (Separate X-Axis)",
    xaxis_title="Date",
    yaxis_title="MAE",
    legend_title="Model"
)

fig.update_xaxes(tickformat="%b %d", tickangle=45)
fig.show()

In [ ]:
df_result = pd.read_csv("../data/pred_result.csv")

In [ ]:
df_result.groupby("dteday").apply(lambda x: mean_absolute_error(x["cnt"], x['cnt_pred']))

In [ ]:
# Compare mean MAE per model
print(results.groupby('model')['mae'].mean().round(3))



In [ ]:
# 2. Split to Train and Test
train_size = int(len(hourly_cnt) * 0.8)
train, test = hourly_cnt[:train_size], hourly_cnt[train_size:]

# 3. Apply Seasonal Differencing
seasonal_lag = 24
train_diff = train.diff(seasonal_lag).dropna()
check_stationarity(train_diff)

# 4. Fit SARIMA Model (Example)
model = ARIMA(train_diff, order=(2, 0, 3))
results = model.fit()

# 5. Forecast Train and Test
train_forecast_diff = results.fittedvalues
test_forecast_diff = results.forecast(steps=len(test))

# 6. Inverse Transformation for Train
train_forecast_original = pd.Series(index=train_diff.index)

# Initialize the first 24 values
for i in range(seasonal_lag):
    train_forecast_original[train_diff.index[i]] = train[train_diff.index[i]]

# Inverse transform the rest of the values
for i in range(seasonal_lag, len(train_diff)):
    train_forecast_original[train_diff.index[i]] = train_forecast_original[train_diff.index[i - seasonal_lag]] + train_forecast_diff[i-seasonal_lag]

# 7. Inverse Transformation for Test
test_forecast_original = pd.Series(index=test.index)

# Initialize the first 24 values using the last 24 values from train
for i in range(seasonal_lag):
    test_forecast_original[test.index[i]] = train[-seasonal_lag + i] + test_forecast_diff[i]

# Inverse transform the rest of the values
for i in range(seasonal_lag, len(test_forecast_diff)):
    test_forecast_original[test.index[i]] = test_forecast_original[test.index[i - seasonal_lag]] + test_forecast_diff[i]

# 8. Plot Train and Test Predictions
plt.figure(figsize=(15, 8))
plt.plot(train, label='Train Data')
plt.plot(train_forecast_original, label='Train Forecast', color='green')
plt.show()
plt.figure(figsize=(15, 8))

plt.plot(test, label='Test Data')
plt.plot(test_forecast_original, label='Test Forecast', color='red')
plt.legend()
plt.show()
mae_train = mean_absolute_error(train[24:], train_forecast_original)
mae_test = mean_absolute_error(test, test_forecast_original)
print(f"Train mae: {mae_train}")
print(f"Test mae: {mae_test}")

In [ ]:

pd.to_datetime(train.index.max()).dt.date

In [ ]:
test.index.min()

In [ ]:
RMSE = np.sqrt(mean_squared_error(train.iloc[24:], train_forecast_original))
print(RMSE)

In [ ]:
test_forecast_original.head()

In [ ]:
test.head()